In [14]:
import pandas as pd
import os
from tqdm import *
import numpy as np

import re

In [3]:
# variables 
bus_stops = [43901.0]
encoding = 'latin1'

p = r"C:\Data\Transformiert"
n_p = os.path.join(p, 'interim')
n_a = os.path.join(p, 'all')

if not os.path.exists(n_p):
    os.makedirs(n_p)
if not os.path.exists(n_a):
    os.makedirs(n_a)    

files = os.listdir(p)
files = [f for f in files if '.csv' in f]

n_p_files = os.listdir(n_p)
for f in n_p_files:
    os.remove(os.path.join(n_p,f))
    
usecols_names = ['IBIS', 'DatumAn', 'ZeitAn', 'HSTNr', 'HSTName', 'Ein', 'Aus', 'X', 'Y']

In [4]:
# Gets the column Ids 

f_n = 0 
for i in range(0, len(files)):
    if f_n % 10 == 0:
        print f_n
    try:
        df = pd.read_csv(os.path.join(p, files[f_n]), sep=';', index_col=False)
        cols = df.columns
        cols_dict = {}
        for i, col in enumerate(cols):
            cols_dict[col] = i

        usecols_ids = []
        for n in usecols_names:
            usecols_ids.append(cols_dict[n])
        break
    except KeyError:
        if f_n > len(files):
            break
        f_n += 1

0


In [8]:
df

,IBIS,HSTNr,HSTName,X,Y,DatumAn,ZeitAn,Ein,Aus


In [7]:
f_n = 0
for i in range(0, len(files)):
    try:
        df = pd.read_csv(os.path.join(p, files[f_n]), sep=';', index_col=False, usecols=usecols_ids, encoding=encoding)
        df.dropna(inplace=True)
        df = df[df['HSTNr'].isin(bus_stops)]
        total = len(files)
        break
    except Exception as e:
        print e
        if i == 20:
            break
        f_n += 1 

In [9]:
# file number - iterate until a nice file is found
f_n = 0
for i in range(0, len(files)):
    try:
        df = pd.read_csv(os.path.join(p, files[f_n]), sep=';', index_col=False, usecols=usecols_ids, encoding=encoding)
        df.dropna(inplace=True)
        df = df[df['HSTNr'].isin(bus_stops)]
        total = len(files)
        break
    except:
        f_n += 1 

# all file number    
a_f_n = 0

for i, f in tqdm(enumerate(files[f_n:]), total = total-1):
    try:
        df_new = pd.read_csv(
                os.path.join(p, f), sep=';', index_col=False, usecols=usecols_ids, encoding=encoding)
        df_new.dropna(inplace=True)

        # Select only Sophienstrasse (43901.0)
        df_new = df_new[df_new['HSTNr'].isin(bus_stops)]
        df = df.append(df_new)
        if i % 30 == 0:
            df.to_csv(os.path.join(n_p, 'all' + str(f_n) + '.csv'), sep= ';', encoding=encoding)
            df = df[df.index == None]
            f_n += 1
    except Exception as e:
        print Exception
        print e
        break

15976it [02:30, 106.44it/s]                                                                                            


In [69]:
df_name = 'all'
for stop in bus_stops:
    df_name = df_name + '_' + str(stop)[:-2]
df_name += '.csv'

files = os.listdir(n_p)
df = pd.read_csv(os.path.join(n_p, files[0]), sep=';', index_col=0, encoding=encoding)
for f in tqdm(files[1:]):
    df = df.append(pd.read_csv(os.path.join(n_p, f), sep=';', encoding=encoding))

# Extract? the bus route and the bus target 
df['Linie'] = df['IBIS'].str.extract('(Linie (E|N)*[0-9]{1,2})')[0]
df['Ziel'] = df['IBIS'].str.extract('Z: ([0-9]*)')
df['Nachtbus'] = df['Linie'].str.match('N')
df['E (Ersatz?)'] = df['Linie'].str.match('E')
df['Normal'] = df['Linie'].str.match('(E|N)')

def normal(x):
    if x is None:
        return np.nan
    return not x

df['Normal'] = df['Normal'].apply(lambda x: normal(x))
    
df[usecols_names].to_csv(os.path.join(n_a, df_name), index=False, encoding=encoding)

100%|███████████████████████████████████████████████████████████████████████████████| 532/532 [00:03<00:00, 157.49it/s]
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.


In [70]:
df

,Aus,DatumAn,Ein,HSTName,HSTNr,IBIS,Unnamed: 0,X,Y,ZeitAn,Linie,Ziel,Nachtbus,E (Ersatz?),Normal
0,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"H: Sophienstraße (0), H: Sophienstraße (0), H:...",43.0,3406.845,5758.884,05:50:19,NaN,NaN,NaN,NaN,False
1,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"H: Sophienstraße (0), Z: 431 (0), H: Linie 14 ...",63.0,3406.850,5758.886,06:29:19,Linie 14,431,False,False,True
2,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"H: Sophienstraße (0), H: Linie 14 (0), H: Hans...",110.0,3406.850,5758.886,07:50:27,Linie 14,NaN,False,False,True
3,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"Z: 431 (0), H: Sophienstraße (0), H: Linie 14 ...",130.0,3406.873,5758.872,08:27:14,Linie 14,431,False,False,True
4,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"H: Sophienstraße (0), H: Sophienstraße (0), H:...",175.0,3406.882,5758.866,09:50:47,Linie 14,NaN,False,False,True
5,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"H: Sophienstraße (0), Z: 431 (0), H: Sophienst...",195.0,3406.874,5758.871,10:23:53,Linie 14,431,False,False,True
6,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"H: Sophienstraße (0), H: Sophienstraße (0), H:...",240.0,3406.880,5758.866,11:52:01,Linie 14,NaN,False,False,True
7,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"H: Sophienstraße (0), Z: 431 (0), H: Sophienst...",258.0,3406.878,5758.874,12:27:12,Linie 14,431,False,False,True
8,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"H: Sophienstraße (0), H: Linie 14 (0), H: Hans...",305.0,3406.875,5758.868,13:52:28,Linie 14,NaN,False,False,True
9,0.0,12.04.2017,0.0,Sophienstraße,43901.0,"Z: 431 (0), H: Sophienstraße (0), H: Linie 14 ...",325.0,3406.880,5758.876,14:25:33,Linie 14,431,False,False,True


In [57]:
df['Linie'].unique()

array([nan, u'Linie 14', u'Linie N84', u'Linie E11', u'Linie 11',
       u'Linie N82', u'Linie E14', u'Linie 8', u'Linie 22', u'Linie 1',
       u'Linie 9', u'Linie E6', u'Linie 12', u'Linie E17', u'Linie E10',
       u'Linie N83', u'Linie 10', u'Linie E84', u'Linie 19', u'Linie N81',
       u'Linie 33', u'Linie 5'], dtype=object)